In [2]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [3]:
# Step 1: Define fuzzy variables
# Inputs
num_junctions = ctrl.Antecedent(np.arange(1, 11, 1), 'num_junctions')
length_junction = ctrl.Antecedent(np.arange(0, 501, 1), 'length_junction')
distance_between = ctrl.Antecedent(np.arange(0, 1001, 1), 'distance_between')
traffic_density = ctrl.Antecedent(np.arange(0, 101, 1), 'traffic_density')
avg_speed = ctrl.Antecedent(np.arange(0, 61, 1), 'avg_speed')

In [4]:
# Outputs
red_duration = ctrl.Consequent(np.arange(10, 101, 1), 'red_duration')
green_duration = ctrl.Consequent(np.arange(10, 101, 1), 'green_duration')
yellow_duration = ctrl.Consequent(np.arange(3, 11, 1), 'yellow_duration')

In [5]:
# Step 2: Define membership functions
# Membership for number of junctions
num_junctions['few'] = fuzz.trimf(num_junctions.universe, [1, 1, 5])
num_junctions['moderate'] = fuzz.trimf(num_junctions.universe, [1, 5, 10])
num_junctions['many'] = fuzz.trimf(num_junctions.universe, [5, 10, 10])

In [6]:
# Membership for length of junction
length_junction['short'] = fuzz.trimf(length_junction.universe, [0, 0, 250])
length_junction['medium'] = fuzz.trimf(length_junction.universe, [0, 250, 500])
length_junction['long'] = fuzz.trimf(length_junction.universe, [250, 500, 500])

In [7]:
# Membership for distance between junctions
distance_between['close'] = fuzz.trimf(distance_between.universe, [0, 0, 500])
distance_between['moderate'] = fuzz.trimf(distance_between.universe, [0, 500, 1000])
distance_between['far'] = fuzz.trimf(distance_between.universe, [500, 1000, 1000])


In [8]:
# Membership for traffic density
traffic_density['low'] = fuzz.trimf(traffic_density.universe, [0, 0, 50])
traffic_density['medium'] = fuzz.trimf(traffic_density.universe, [0, 50, 100])
traffic_density['high'] = fuzz.trimf(traffic_density.universe, [50, 100, 100])

In [9]:
# Membership for average speed
avg_speed['slow'] = fuzz.trimf(avg_speed.universe, [0, 0, 30])
avg_speed['moderate'] = fuzz.trimf(avg_speed.universe, [0, 30, 60])
avg_speed['fast'] = fuzz.trimf(avg_speed.universe, [30, 60, 60])

In [10]:
# Membership for red light duration
red_duration['short'] = fuzz.trimf(red_duration.universe, [10, 10, 40])
red_duration['medium'] = fuzz.trimf(red_duration.universe, [10, 40, 80])
red_duration['long'] = fuzz.trimf(red_duration.universe, [40, 80, 100])

In [11]:
# Membership for green light duration
green_duration['short'] = fuzz.trimf(green_duration.universe, [10, 10, 40])
green_duration['medium'] = fuzz.trimf(green_duration.universe, [10, 40, 80])
green_duration['long'] = fuzz.trimf(green_duration.universe, [40, 80, 100])

In [12]:
# Membership for yellow light duration
yellow_duration['short'] = fuzz.trimf(yellow_duration.universe, [3, 3, 7])
yellow_duration['medium'] = fuzz.trimf(yellow_duration.universe, [3, 7, 10])
yellow_duration['long'] = fuzz.trimf(yellow_duration.universe, [7, 10, 10])

In [13]:
# Step 2: Define membership labels for iteration
num_junctions_labels = ['few', 'moderate', 'many']
length_junction_labels = ['short', 'medium', 'long']
distance_between_labels = ['close', 'moderate', 'far']
traffic_density_labels = ['low', 'medium', 'high']
avg_speed_labels = ['slow', 'moderate', 'fast']


In [14]:
# Modify rule creation to specify each consequent individually in a list format
rules = []

for nj_label in num_junctions_labels:
    for lj_label in length_junction_labels:
        for db_label in distance_between_labels:
            for td_label in traffic_density_labels:
                for as_label in avg_speed_labels:
                    # Define the conditions for the rule
                    condition = (num_junctions[nj_label] &
                                 length_junction[lj_label] &
                                 distance_between[db_label] &
                                 traffic_density[td_label] &
                                 avg_speed[as_label])
                    
                    # Define default actions (customizable)
                    if td_label == 'high' and as_label == 'slow':
                        actions = [
                            green_duration['long'],
                            red_duration['short'],
                            yellow_duration['medium']
                        ]
                    elif td_label == 'low' and as_label == 'fast':
                        actions = [
                            red_duration['long'],
                            green_duration['short'],
                            yellow_duration['short']
                        ]
                    else:
                        actions = [
                            green_duration['medium'],
                            red_duration['medium'],
                            yellow_duration['medium']
                        ]
                    
                    # Create and append the rule for each set of actions
                    rule = ctrl.Rule(condition, actions)
                    rules.append(rule)


In [15]:
# Step 4: Create a control system with all rules
traffic_light_ctrl = ctrl.ControlSystem(rules)
traffic_light_simulation = ctrl.ControlSystemSimulation(traffic_light_ctrl)

In [16]:
# Step 5: Input values and compute output
traffic_light_simulation.input['num_junctions'] = 3
traffic_light_simulation.input['length_junction'] = 350
traffic_light_simulation.input['distance_between'] = 400
traffic_light_simulation.input['traffic_density'] = 70
traffic_light_simulation.input['avg_speed'] = 20

traffic_light_simulation.compute()

In [17]:
# Output the results
print(f"Red Light Duration: {traffic_light_simulation.output['red_duration']:.2f} seconds")
print(f"Green Light Duration: {traffic_light_simulation.output['green_duration']:.2f} seconds")
print(f"Yellow Light Duration: {traffic_light_simulation.output['yellow_duration']:.2f} seconds")

Red Light Duration: 42.06 seconds
Green Light Duration: 53.23 seconds
Yellow Light Duration: 6.61 seconds
